### Original SImulation Approach 
- divide into 1st cycle + subsequent cycles

##### Key Assumptions

- will only be looking at ships that arrive at Port of Long Beach (to limit scope) 
- thus filtered Julien's optimization output data to ships that have cargo scheduled for Long Beach
- ~~HOWEVER, the ship length will still determine whether or not the vesssel can park at the berth~~ 
    - decided to keep port capacity constant at first (to simplify the problem) 
    - this would keep it a single server model

- Terminal Capacity is multiplied by 70% and the floor of the integer is taken (to be conservative) 
- Time per TEU / per crane = (1/(26.5 * 1.54)) = 0.0245
    - note that avg crane makes 26.5 moves (containers) per hour
    - and each container is avg 1.54 TEU / container 
    - utilization is given by assuming that ALL the cranes at the port work simultaneously

   
- Create an SHIP ARRIVAL list! (so make it deterministic! and loop through this list!) (just like the patient list)



#### One Approach - Generation of list of ships (scheduled arrival of ships) 
- code is implemented below 
- also tried this approach, where I generate a list of ships before hand based on the mean,std of the ship voyage. 
- Then sort the entire dataframe by time, and have each ship arrive at the port via a Poisson distribution / exponential time 
    - this would simplify it so that we could loop through the vessel list in the simulation body
- sample without replacement  
- get a list of ships to "arrive" at the port


In [3]:
import numpy as np
import pandas as pd

###### Dataset Import and Filtering

In [12]:
port_capacity = pd.read_csv("portcapacity.csv")

In [13]:
## Will only look at Port of Long Beach + Terminals with Rail System
terminal_capacity = pd.read_csv("terminalcapacity.csv")
lb_port = terminal_capacity[(terminal_capacity['Port']=="Long Beach") & (terminal_capacity['Rail System']==1)]
# lb_port

In [14]:
orig = lb_port[['Name','Port','Berth','Yard','Rail System']]

## Take the floor of the multiplied output
## only reduce the crane #s + the container capacity 
reduced = np.floor(lb_port[['Crane_Panamax','Crane_PostPX','Crane_SuperPX','ContainerCap']].multiply(0.7, axis=1))
reduced_lb = pd.concat([orig, reduced], axis=1) 
reduced_lb['TotalCrane'] = reduced_lb[['Crane_Panamax','Crane_PostPX','Crane_SuperPX']].sum(axis=1)
reduced_lb['Avail_Berth'] = reduced_lb['Berth']   ## Necessary for later calculations
reduced_lb = reduced_lb.sort_values(by=['Name']).reset_index(drop=True)

In [15]:
reduced_lb

,Name,Port,Berth,Yard,Rail System,Crane_Panamax,Crane_PostPX,Crane_SuperPX,ContainerCap,TotalCrane,Avail_Berth
0,Pier A,Long Beach,3600,200,1,0.0,0.0,7.0,84.0,7.0,3600
1,Pier E,Long Beach,2750,154,1,0.0,0.0,5.0,64.0,5.0,2750
2,Pier F,Long Beach,2750,88,1,0.0,3.0,1.0,64.0,4.0,2750
3,Pier G,Long Beach,6379,246,1,1.0,7.0,1.0,149.0,9.0,6379
4,Pier J,Long Beach,5900,256,1,0.0,4.0,6.0,138.0,10.0,5900
5,Pier T,Long Beach,5000,385,1,0.0,0.0,9.0,117.0,9.0,5000


###### Vessel Type and Length Inference from max TEU values 

In [16]:
def vessel_length(max_teu):
    ## SuperPX crane
    if max_teu > 14400:   
        v_length = 1300
        v_type = 2
    ## SuperPX crane
    elif max_teu > 10000:    
        v_length = 1200 
        v_type = 2
    ## Post PX crane
    elif max_teu > 5000:
        v_length = 1100
        v_type = 1
    ## Panamax Crane
    else: 
        v_length = 965
        v_type = 0
    return v_length, v_type

In [17]:
vessels = pd.read_csv("longbeach.csv")

## Apply the vessel type & length above and unpack the tuple to append to original dataframe
## Reference: https://stackoverflow.com/questions/56775952/unpacking-a-tuple-on-multiple-columns-of-a-dataframe-from-series-apply
vessels[['length','v_type']] = vessels['teu'].apply(lambda x: vessel_length(x)).tolist()

## Exclude vessles without any stops (did not cross the pacific)
vessels = vessels[vessels['stops']!=0].reset_index(drop=True)
vessels['mean_hr'] = vessels['mean'] * 24
vessels['stddev_hr'] = vessels['stddev'] * 24

## decision variable
## Set 0 if able to be picked.....1 if not (still en route to port)
vessels['cycle_complete'] = 1


In [18]:
vessels.head()

,shipname,teu,maxspd,mean,stddev,stops,longbeach_cargo,avg_speed,length,v_type,mean_hr,stddev_hr,cycle_complete
0,EVERFINE,11888,23,50.262613,2.513131,3,614,20.5,1200,2,1206.302712,60.315136,1
1,CMACGMTITAN,11388,22,50.441776,2.522089,3,643,20.0,1200,2,1210.602622,60.530131,1
2,CMACGMRIGOLETTO,9415,22,41.947906,2.097395,5,1075,20.0,1100,1,1006.749743,50.337487,1
3,APLVANCOUVER,9326,21,45.434334,2.271717,2,1637,19.5,1100,1,1090.424016,54.521201,1
4,NYKVESTA,9012,21,48.502162,2.425108,2,1637,19.5,1100,1,1164.051898,58.202595,1


#### Poission Distribution + Exponential Interarrival Times

In [20]:
### Constants
hours_per_teu = 1/(26.5*1.54) 

poisson_lambda = 1.14   # example rate = 1.14 vessels /day
exp_mean = 1/poisson_lambda
exp_mean

0.8771929824561404

#### Under Infinite Capacity

In [32]:
## For the first cycle (first 27 vessels, just sample without replacement)
## ALL Ships arrive according to the Poisson process (meaning interarrival time follows the exponential distribution)
## Units are in days
M = 100

berth_tol = 100
vessel_array = np.zeros((4, M))
shipname_ls = []
terminal_name_ls = []


n_vessels = 27
vessel_arrival_time = 0

vessel_arrival_index = 0
cargo_load_index = 1
service_duration_index = 2
vessel_departure_index = 3

for v in range(n_vessels):
    ## Record the Arrival Time of each vessel
    r1 = np.random.uniform(0,1) 
    interarrival_time = -exp_mean * np.log(r1)                    # given by the Poission Distribution
    vessel_arrival_time += interarrival_time
    
    vessel_array[vessel_arrival_index][v] = vessel_arrival_time
    
    
    ## Pick the necessary vessel
    ## For the first cycle, just pick the vessels sequentially
    if v < len(vessels): 
        vessel_name = vessels['shipname'][v]
        shipname_ls.append(vessel_name)
        vessels.loc[v,['cycle_complete']] = 0  ## 
        
        ## Assume infinite capacity... meaning that the vessel can go into ANY terminal + all cranes will work on the single vessel

        vessel_array[cargo_load_index][v] = vessels['longbeach_cargo'][v]
        service_duration = (vessels['longbeach_cargo'][v] * hours_per_teu) / (24 * reduced_lb['TotalCrane'].sum()) ## (44 cranes + also convert to days)

        vessel_array[service_duration_index][v] = service_duration
        vessel_departure_time = vessel_arrival_time + service_duration
        vessel_array[vessel_departure_index][v] = vessel_departure_time   
        
    elif: 
        ## After selecting each row of the vessels dataframe once, would need to sample based on if the vessel has 
        ## reached its period_duration or not 
        
        
        
#         ## Check which terminal it can go into (based on berth length)
#         ## Fill up the minimum length berth first! 
#         min_berth_idx = reduced_lb['Avail_Berth'].idxmin()
#         if vessels.loc[v, 'length'] < reduced_lb.loc[min_berth_idx, 'Avail_Berth']:   # If the vessel length fits into berth
#             total_length_occupied = reduced_lb.loc[min_berth_idx, 'Avail_Berth'] - ((vessels.loc[v, 'length']) + berth_tol)
#             reduced_lb.loc[min_berth_idx, 'Avail_Berth'] = total_length_occupied
#             terminal_name_ls.append(reduced_lb.loc[min_berth_idx, 'Name'])

#             ## Start Cargo Operations
#             cargo = vessels.loc[v, 'longbeach_cargo']
#             vessel_array[cargo_load_index][v] = cargo
#             cargo_duration = time_per_teu * cargo / reduced_lb.loc[min_berth_idx, 'TotalCrane']  ## units in hours
#             vessel_array[cargo_duration_index][v] = cargo_duration / 24    ## Convert unit to days! 
            
#             ## End Cargo Operations
#             vessel_departure_time = vessel_arrival_time + cargo_duration
#             vessel_array[vessel_departure_index][v]  = vessel_departure_time
#             ## - Add back in the Berth Length
            
            
            
    
#     ## Do a random sampling (BUT take into account periodicity!! )
#     else: 
#         a = vessels.sample(n=len(vessels), replace=False, random_state=v)
        
        
#     print(a)
# vessel_ls
# vessels.head(6)
a = pd.DataFrame((vessel_array.T), columns=['arrival_time','cargo','service_duration','departure_time'])
a.head(40)


,arrival_time,cargo,service_duration,departure_time
0,0.024748,614.0,0.014247,0.038995
1,0.783171,643.0,0.014920,0.798092
2,1.858522,1075.0,0.024945,1.883467
3,2.449715,1637.0,0.037986,2.487700
4,2.706525,1637.0,0.037986,2.744511
5,3.939581,1763.0,0.040909,3.980490
6,4.237501,3955.0,0.091773,4.329274
7,4.378138,4244.0,0.098479,4.476617
8,6.234001,4484.0,0.104048,6.338049
9,6.998571,4516.0,0.104791,7.103362


In [82]:
vessels.sample(replace=False)

,shipname,teu,maxspd,mean,stddev,stops,longbeach_cargo,avg_speed,cycle_complete,length,v_type,mean_hr,stddev_hr
17,COSCOPORTUGAL,13386,24,35.82944,1.791472,2,12177,21.0,0,1200,2,859.906559,42.995328


In [36]:
## decision variable
## SEt 0 if able to be picked.....1 if 
vessels['cycle_complete'] = 0 

In [19]:

ship_df = []
for i in range(1):
    test = vessels.sample(n=len(vessels), replace=False, random_state=i)
    ship_df.append(test)

output = pd.concat(ship_df)

In [22]:
output

,shipname,teu,maxspd,mean,stddev,stops,longbeach_cargo,avg_speed,length,v_type,mean_hr,stddev_hr
2,CMACGMRIGOLETTO,9415,22,41.947906,2.097395,5,1075,20.0,1100,1,1006.749743,50.337487
24,COSCOSHIPPINGDENALI,14566,23,38.827048,1.941352,2,14566,20.5,1300,2,931.849152,46.592458
14,CSCLSOUTHCHINASEA,10036,24,31.112289,1.555614,1,9986,21.0,1200,2,746.694928,37.334746
17,COSCOPORTUGAL,13386,24,35.829440,1.791472,2,12177,21.0,1200,2,859.906559,42.995328
5,CMACGMMEXICO,15072,22,54.976762,2.748838,2,1763,20.0,1300,2,1319.442294,65.972115
11,EVERFOREVER,12010,23,42.859818,2.142991,2,7522,20.5,1200,2,1028.635640,51.431782
23,CMACGMJ.MADISON,14414,22,41.466573,2.073329,1,14414,20.0,1300,2,995.197746,49.759887
13,APLCOLUMBUS,9326,21,16.827785,0.841389,1,9285,19.5,1100,1,403.866850,20.193343
19,APLSANTIAGO,9326,21,36.411717,1.820586,2,13830,19.5,1100,1,873.881213,43.694061
20,CMACGMCORTEREAL,13830,24,39.604441,1.980222,1,13830,21.0,1200,2,950.506575,47.525329


#### Generation of list of ships (scheduled arrival of ships) 
- also tried this approach, where I generate a list of ships before hand based on the mean,std of the ship voyage. 
- Then sort the entire dataframe by time, and have each ship arrive at the port via a Poisson distribution / exponential time 
    - this would simplify it so that we could loop through the vessel list in the simulation body
- sample without replacement  
- get a list of ships to "arrive" at the port


In [252]:
scheduled_arrival = vessels

In [254]:
num_of_samples = 10
start = 0
total_ls = []

for i in range(len(scheduled_arrival)):
    each_ship_ls = []
    each_time = 0
    mean = scheduled_arrival['mean'].iloc[i]
    std = scheduled_arrival['stddev'].iloc[i]    
    
    for j in range(num_of_samples):
        delta_time = np.random.normal(mean, std)
        each_time += delta_time
        
        each_ship_ls.append(each_time)
    
    total_ls.append(each_ship_ls)

cum_arrival_time = list(np.concatenate(total_ls).flat)

In [255]:
arrival_ls = pd.concat([scheduled_arrival] * num_of_samples)
arrival_ls = arrival_ls.sort_index()
arrival_ls['cum_time'] = list(np.concatenate(total_ls).flat)

In [263]:
vessels

,shipname,teu,maxspd,mean,stddev,stops,longbeach_cargo,avg_speed,length,v_type,mean_hr,stddev_hr
0,EVERFINE,11888,23,50.262613,2.513131,3,614,20.5,1200,2,1206.302712,60.315136
1,CMACGMTITAN,11388,22,50.441776,2.522089,3,643,20.0,1200,2,1210.602622,60.530131
2,CMACGMRIGOLETTO,9415,22,41.947906,2.097395,5,1075,20.0,1100,1,1006.749743,50.337487
3,APLVANCOUVER,9326,21,45.434334,2.271717,2,1637,19.5,1100,1,1090.424016,54.521201
4,NYKVESTA,9012,21,48.502162,2.425108,2,1637,19.5,1100,1,1164.051898,58.202595
5,CMACGMMEXICO,15072,22,54.976762,2.748838,2,1763,20.0,1300,2,1319.442294,65.972115
6,OOCLTOKYO,8063,25,33.759822,1.687991,3,3955,21.5,1100,1,810.235727,40.511786
7,CMACGMPANAMA,15072,22,59.064515,2.953226,2,4244,20.0,1300,2,1417.548371,70.877419
8,COSCONETHERLANDS,13386,24,37.154381,1.857719,3,4484,21.0,1200,2,891.705138,44.585257
9,CSCLYELLOWSEA,10036,24,36.806488,1.840324,2,4516,21.0,1200,2,883.355713,44.167786


In [261]:
# arrival_ls.sort_values(by=['cum_time']).head(50)

In [262]:
terminal_capacity

,Name,Port,Berth,Yard,Crane_Panamax,Crane_PostPX,Crane_SuperPX,Rail System,ContainerCap
0,Terminal 18,Seattle-Tacoma,4440,196,0,3,7,1,190.016949
1,Terminal 25/30,Seattle-Tacoma,2700,70,3,0,3,0,115.550848
2,Terminal 46,Seattle-Tacoma,2300,82,0,3,3,0,98.432203
3,Husky,Seattle-Tacoma,2700,93,0,4,0,1,126.632769
4,East Sitcum,Seattle-Tacoma,1100,54,4,0,0,1,51.591128
5,West Sitcum,Seattle-Tacoma,2200,135,1,4,0,0,103.182256
6,Washington United,Seattle-Tacoma,2600,123,0,4,2,1,121.942667
7,Pierce County,Seattle-Tacoma,2807,140,0,0,7,1,131.651179
8,West Basin 1,Los Angeles,2500,132,0,0,10,1,87.841992
9,West Basin 2,Los Angeles,2500,186,0,5,0,1,87.841992
